In [3]:
###<进阶> 5.2.1 从字典类型加载特征
from sklearn.feature_extraction import DictVectorizer

#类 DictVectorizer 可用于将标准的Python字典（dict）对象列表的要素数组转换为 scikit-learn 估计器使用的 NumPy/SciPy 表示形式。
#类 DictVectorizer 实现了 “one-of-K” 或 “one-hot” 编码，用于分类（也称为标称，离散）特征。

#在下面的例子，”城市” 是一个分类属性，而 “温度” 是传统的数字特征，对“城市”属性进行了one-hot编码:
measurements = [
    {'city': 'Dubai', 'temperature': 33.},
    {'city': 'London', 'temperature': 12.},
    {'city': 'San Francisco', 'temperature': 18.},
]

vec = DictVectorizer()
vec.fit_transform(measurements).toarray()


array([[ 1.,  0.,  0., 33.],
       [ 0.,  1.,  0., 12.],
       [ 0.,  0.,  1., 18.]])

In [4]:
vec.get_feature_names_out()

array(['city=Dubai', 'city=London', 'city=San Francisco', 'temperature'],
      dtype=object)

In [15]:
###<进阶> 5.2.2 特征哈希（相当于一种降维技巧）
#class sklearn.feature_extraction.FeatureHasher(n_features=1048576, *, input_type='dict', dtype=<class 'numpy.float64'>, alternate_sign=True)[source]¶
#此类将符号特征名称（字符串）序列转换为scipy.sparse矩阵，使用哈希函数计算与名称对应的矩阵列。所使用的哈希函数是Murmurhash3的32位签名版本。

## input_type是字典结构时
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=10)  #输出特征维度
D = [{'dog': 1, 'cat':2, 'elephant':4},{'dog': 2, 'run': 5}]  #hash字典结构的，2个样本
f = h.transform(D)
arr_hash = f.toarray()
arr_hash


array([[ 0.,  0., -4., -1.,  0.,  0.,  0.,  0.,  0.,  2.],
       [ 0.,  0.,  0., -2., -5.,  0.,  0.,  0.,  0.,  0.]])

In [16]:
# 特征hash其实就是对所有输入特征分别hash，再对结果合并
# 我们把{'dog': 1, 'cat':2, 'elephant':4}样本拆分成3个样本理解，每次hash只输入单个特征的样本
h = FeatureHasher(n_features=10)  #输出特征维度
D1 = [{'dog': 1}]  
D2 = [{'cat':2}]  
D3 = [{'elephant':4}]  

f1 = h.transform(D1)   #hash特征dog，hash值如下
arr1 = f1.toarray()
arr1

array([[ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [17]:
f2 = h.transform(D2)  #hash特征cat，hash值如下
arr2 = f2.toarray()
arr2

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]])

In [18]:
f3 = h.transform(D3) #hash特征elephant，hash值如下
arr3 = f3.toarray()
arr3

array([[ 0.,  0., -4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [19]:
arr_result = arr1 + arr2 + arr3   # 3个特征分别hash后拼成一条样
arr_result                        # 对比可知样本hash后的特征是每个特征hash后的汇总，即arr_resut = arr_hash

array([[ 0.,  0., -4., -1.,  0.,  0.,  0.,  0.,  0.,  2.]])

In [20]:
# input_type=“string”时，输入必须是字符串的可迭代项上的可迭代值：
h = FeatureHasher(n_features=8, input_type="string")
raw_X = [["dog", "cat", "snake"], ["snake", "dog"], ["cat", "bird"]]
f = h.transform(raw_X)
f.toarray()

array([[ 0.,  0.,  0., -1.,  0., -1.,  0.,  1.],
       [ 0.,  0.,  0., -1.,  0., -1.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
## 总结

#1、hash函数的概念，请复习数据结构
#2、对特征空间很大的特征（例如one-hot展开空间很大，上万/十万/百万特征），为了节省内存或提高训练效率，可特征hash进行降维，当然这也会损失一些信息。
#3、其他文本、图像处理特征处理暂未列出，更多请参考 https://scikit-learn.org/stable/modules/feature_extraction.html